## Setup

In [71]:
import urllib.request
import zipfile
import pandas as pd
from urllib.parse import urlencode
import json

### Define Data

In [2]:
DATA_DIR = './data/'
ZIPS_URL = 'http://www2.census.gov/geo/docs/maps-data/data/gazetteer/2017_Gazetteer/2017_Gaz_zcta_national.zip'
ZIPS_FP = DATA_DIR + 'zips-2017.zip'
ZHVI_URL = 'http://files.zillowstatic.com/research/public/Zip/Zip_Zhvi_SingleFamilyResidence.csv'
ZHVI_FP = DATA_DIR + 'sfr.csv'

### (Optional) Refresh Data

In [3]:
urllib.request.urlretrieve(ZIPS_URL, ZIPS_FP);

In [7]:
urllib.request.urlretrieve(ZHVI_URL, ZHVI_FP);

## Home Prices

In [5]:
df = pd.read_csv(ZHVI_FP)
counties = ['San Francisco', 'San Mateo', 'San Jose']
df = df[df['CountyName'].isin(counties)]
zips = df['RegionName']

## Schools

In [7]:
df2 = pd.read_csv(ZIPS_FP, sep='\t')
df2 = df2[df2['GEOID'].isin(zips.values)]

In [51]:
base = 'https://www.greatschools.org/search/search.page?'
params = {
    'lat': '37.554298',
    'lon': '-122.496632',
    'zip': '94037',
    'state': 'CA'
}
url = base + urlencode(query)
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:58.0) Gecko/20100101 Firefox/58.0",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "accept-encoding": "gzip,deflate,br",
    "accept-language": "en-US,en;q=0.5",
}

In [54]:
req = urllib.request.Request(url, None, headers)
with urllib.request.urlopen(url) as res:
    html = str(res.read())

In [78]:
raw = html.find('gon.map_points')
start = html.find('[',raw)
end = html.find(']',raw)
data = html[start:end+1]
schools = json.loads(data)
rated = [i for i in schools if i['gsRating'] > 0]

In [79]:
rated

[{'city': 'Pacifica',
  'communityRatingStars': 5,
  'fitScore': 0,
  'gradeRange': 'K-8',
  'gsRating': 9,
  'id': 6884,
  'lat': 37.59849548339844,
  'lng': -122.49832153320312,
  'maxFitScore': 0,
  'name': 'Cabrillo Elementary School',
  'numReviews': 129,
  'okFit': False,
  'on_page': True,
  'preschool': False,
  'profileUrl': '/california/pacifica/6884-Cabrillo-Elementary-School/',
  'reviewUrl': '/california/pacifica/6884-Cabrillo-Elementary-School/#Reviews',
  'schoolType': 'Public district',
  'state': 'ca',
  'street': '601 Crespi Drive',
  'strongFit': False,
  'zillowUrl': 'https://www.zillow.com/CA-94044?cbpartner=Great+Schools\\u0026utm_source=GreatSchools\\u0026utm_medium=referral\\u0026utm_campaign=schoolsearch',
  'zipcode': '94044'},
 {'city': 'Pacifica',
  'communityRatingStars': 5,
  'fitScore': 0,
  'gradeRange': 'K-8',
  'gsRating': 9,
  'id': 6892,
  'lat': 37.61370086669922,
  'lng': -122.4847412109375,
  'maxFitScore': 0,
  'name': 'Vallemar Elementary School